In [30]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [31]:
df_green = spark.read.parquet('pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [32]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [33]:
from datetime import datetime

In [34]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [35]:
rows = rdd.filter(filter_outliers).take(10)
row = rows[0]

In [36]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 23, 44, 7), PULocationID=255, total_amount=18.3)

In [37]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [38]:
rdd.filter(filter_outliers).map(prepare_for_grouping).take(10)

[((datetime.datetime(2020, 1, 31, 23, 0), 255), (18.3, 1)),
 ((datetime.datetime(2020, 1, 5, 10, 0), 135), (25.88, 1)),
 ((datetime.datetime(2020, 1, 3, 22, 0), 66), (19.86, 1)),
 ((datetime.datetime(2020, 1, 11, 13, 0), 97), (20.16, 1)),
 ((datetime.datetime(2020, 1, 6, 14, 0), 166), (54.2, 1)),
 ((datetime.datetime(2020, 1, 29, 18, 0), 71), (26.88, 1)),
 ((datetime.datetime(2020, 1, 8, 18, 0), 181), (10.8, 1)),
 ((datetime.datetime(2020, 1, 16, 11, 0), 74), (10.3, 1)),
 ((datetime.datetime(2020, 1, 21, 22, 0), 95), (7.3, 1)),
 ((datetime.datetime(2020, 1, 6, 12, 0), 116), (9.8, 1))]

In [39]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [40]:
rdd.filter(filter_outliers).map(prepare_for_grouping).reduceByKey(
    calculate_revenue
).take(10)

[((datetime.datetime(2020, 1, 29, 18, 0), 71), (99.47999999999999, 5)),
 ((datetime.datetime(2020, 1, 16, 11, 0), 74), (870.0799999999994, 68)),
 ((datetime.datetime(2020, 1, 9, 18, 0), 61), (129.64, 9)),
 ((datetime.datetime(2020, 1, 4, 14, 0), 74), (970.8699999999991, 72)),
 ((datetime.datetime(2020, 1, 27, 14, 0), 177), (100.68, 3)),
 ((datetime.datetime(2020, 1, 19, 14, 0), 75), (650.9599999999999, 41)),
 ((datetime.datetime(2020, 1, 19, 17, 0), 210), (92.6, 2)),
 ((datetime.datetime(2020, 1, 8, 6, 0), 14), (305.59999999999997, 8)),
 ((datetime.datetime(2020, 1, 27, 19, 0), 92), (147.4, 7)),
 ((datetime.datetime(2020, 1, 14, 18, 0), 222), (115.44, 4))]

In [41]:
from collections import namedtuple

In [42]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [43]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [44]:
from pyspark.sql import types

In [45]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [46]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [48]:
df_result.write.parquet('pq/green-revenue')

24/02/27 13:49:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [49]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [50]:
import pandas as pd

In [51]:
rows = duration_rdd.take(10)

In [52]:
df = pd.DataFrame(rows, columns=columns)

In [53]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [54]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [55]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [56]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [57]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|              16.0|
|              21.0|
|             15.85|
|             16.65|
|             85.55|
|              23.2|
|              7.05|
|               9.5|
|               4.5|
|10.149999999999999|
|              21.5|
| 7.800000000000001|
|            125.45|
|             12.85|
|              39.5|
|               2.7|
|              91.0|
|               1.3|
|             23.35|
|              41.5|
+------------------+
only showing top 20 rows

